
# テーマ：日足とその日の初めの分足の足型が一致するかを分析する

# 仕様
* 東証上場銘柄（4,000社以上）を対象とする
  * https://www.jpx.co.jp/markets/statistics-equities/misc/01.html より銘柄一覧をダウンロードし、tickers.csvとして保管しておく。この際、銘柄コードの列名はtickerとする
* 指定した日数の日足と分足（デフォルトでは5分足）の株価をYahooから取得し、日足とその日の初めの分足の足型が一致するかを分析する。結果はresult.csvに出力する。result.csvの各列は以下の通り
  * match_rate：方向性が一致した割合
  * match_count：方向性が一致した数
  * whole_count：日足の数
  * average_volume：出来高の平均値（日足ベース）
  * average_price：株価の平均値（日足の終値ベース）
* Yahooから株価を取得する際、サーバー負荷をかけすぎないようにするため、一定秒数（デフォルトでは5秒）待ってから次の銘柄のデータをダウンロードする。そのため全銘柄ダウンロードするには6時間以上かかる
* 再ダウンロードの機会を減らすため、取得したJSONをファイルに出力

# 出来ていないこと
* ローカルファイル（保管したCSVファイル）を利用した分析

# 今回参考にしたもの
* https://note.com/aiduudia/n/na9ea4f90e255



# 1. Preparation

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/'プログラム（株）'/日足と分足の関連分析

Mounted at /content/drive
/content/drive/My Drive/プログラム（株）


In [ ]:
# Hyper Dashのインストールとサインアップ
!pip install hyperdash
from hyperdash import monitor_cell
!hyperdash signup --github

  Created wheel for hyperdash: filename=hyperdash-0.15.3-cp37-none-any.whl size=28554 sha256=f04bfc24135df30e303577ea61027de332516f2312b38451b6a28dcbd1d9d6ea
  Stored in directory: /root/.cache/pip/wheels/62/5f/af/bbcaeb6570e4904c14fb4c1b70fee559a3788182ce4d104ce7
Successfully built hyperdash
Opening browser, please wait. If something goes wrong, press CTRL+C to cancel.
 SSH'd into a remote machine, or just don't have access to a browser? Open this link in any browser and then copy/paste the provided access token: https://hyperdash.io/oauth/github/start?state=client_cli_manual 
Waiting for Github OAuth to complete.
If something goes wrong, press CTRL+C to cancel.
Access token: xzrrQUrM2WcPNc4R8L7FibWwJ5xeV6ECdWZEC+rUSXI=
Successfully logged in! We also installed: Oox/v9/uFMQXmotqqTxoCom6LFUnugs+p6imDvxcS8E= as your default API key


# 2. Main

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
from statistics import mean

# 定数、グローバル変数
RANGE = '60d' # 何日間のデータを取得するか
DAY_INTERVAL = '1d'
MINUTE_INTERVAL = '5d' # 分足は5分足をデフォルトとする
VOLUME_CRITERIA = 200000 # 出来高がこれより少ないものは無視する
INTERVAL_SECOND = 5 # この秒数待ってから次の銘柄のデータをダウンロードする

JST = timezone(timedelta(hours=+9), 'JST')

# Yahooから株価データを取得
def get_price_data_from_yahoo(ticker):
  json_day, json_minute = None, None

  # 日足取得
  response_day = requests.get('https://query1.finance.yahoo.com/v7/finance/chart/' \
                              + ticker + '?range=' + RANGE + '&interval=' + DAY_INTERVAL + '&indicators=quote&includeTimestamps=true')
  
  if 200 <= response_day.status_code < 300:
    json_day = response_day.json()
    
    # 平均出来高が基準未満であれば以降の処理をスキップ
    volume = 0
    average_volume = 0
    if 'volume' in json_day['chart']['result'][0]['indicators']['quote'][0]:
      volume = json_day['chart']['result'][0]['indicators']['quote'][0]['volume']
      average_volume = mean(d for d in volume if d is not None)
    
    if volume is None or average_volume < VOLUME_CRITERIA:
      return None, None
    
    # データを再利用可能にするため、JSONファイルを保管
    start_date = extract_date(json_day['chart']['result'][0]['timestamp'][0])
    end_date = extract_date(json_day['chart']['result'][0]['timestamp'][-1])
    save_json(json_day, r'./json_data/' + f'{ticker}_day_{start_date}_{end_date}.json')

  # 分足取得
  response_minute = requests.get('https://query1.finance.yahoo.com/v7/finance/chart/' \
                              + ticker + '?range=' + RANGE + '&interval=' + MINUTE_INTERVAL + '&indicators=quote&includeTimestamps=true')
  
  if 200 <= response_minute.status_code < 300:
    json_minute = response_minute.json()

    # データを再利用可能にするため、JSONファイルを保管
    start_date = extract_date(json_minute['chart']['result'][0]['timestamp'][0])
    end_date = extract_date(json_minute['chart']['result'][0]['timestamp'][-1])
    save_json(json_minute, r'./json_data/' + f'{ticker}_minute_{start_date}_{end_date}.json')

  return json_day, json_minute

# タイムスタンプから日付を取得
def extract_date(timestamp):
  date_and_time = str(datetime.fromtimestamp(timestamp))
  date, _ = date_and_time.split()
  return date

# JSONファイル保存
def save_json(json_data, file_path):
  with open(file_path, 'w') as f:
    json.dump(json_data, f, indent=2)

# 株価データ（JSON）をデータフレームに変換
def convert_to_dataframe(json_day, json_minute):
  # 日足
  if json_day['chart']['result'] == None or 'timestamp' not in json_day['chart']['result'][0]:
    return None, None
  df_day = pd.DataFrame()
  df_day['timestamp'] = list(map(datetime.fromtimestamp, json_day['chart']['result'][0]['timestamp'], [JST]*len(json_day['chart']['result'][0]['timestamp'])))
  df_day['open'] = json_day['chart']['result'][0]['indicators']['quote'][0]['open']
  df_day['low'] = json_day['chart']['result'][0]['indicators']['quote'][0]['low']
  df_day['high'] = json_day['chart']['result'][0]['indicators']['quote'][0]['high']
  df_day['close'] = json_day['chart']['result'][0]['indicators']['quote'][0]['close']
  df_day['volume'] = json_day['chart']['result'][0]['indicators']['quote'][0]['volume']
  df_day['positive'] = df_day['open'] < df_day['close']
  df_day['minute_positive'] = None

  # 5分足
  if json_minute['chart']['result'] == None or 'timestamp' not in json_minute['chart']['result'][0]:
    return None, None
  df_minute = pd.DataFrame()
  df_minute['timestamp'] = list(map(datetime.fromtimestamp, json_minute['chart']['result'][0]['timestamp'], [JST]*len(json_minute['chart']['result'][0]['timestamp'])))
  df_minute['open'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['open']
  df_minute['low'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['low']
  df_minute['high'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['high']
  df_minute['close'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['close']
  df_minute['volume'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['volume']

  return df_day, df_minute

# 日足の方向性（陽線/陰線）とその日の初めの5分足の方向性が一致している割合を算出
def analyze_direction(df_day, df_minute, index, df_result):
  # その日の初めの5分足が陽線かどうかをチェック
  for row_day in df_day.itertuples():
    for row_minute in df_minute.itertuples():
      if row_day.timestamp.date() == row_minute.timestamp.date():
        # 出来高0の場合は無視する
        if row_minute.volume <= 0:
          continue
        df_day.at[row_day[0], 'minute_positive'] = row_minute.open < row_minute.close
        break

  df_day['same_direction'] = [1 if r.positive == r.minute_positive else 0 for r in df_day.itertuples()]

  df_result.at[index, 'match_rate'] = df_day['same_direction'].sum() / df_day['same_direction'].count()
  df_result.at[index, 'match_count'] = df_day['same_direction'].sum()
  df_result.at[index, 'whole_count'] = df_day['same_direction'].count()
  df_result.at[index, 'average_volume'] = df_day['volume'].mean()
  df_result.at[index, 'average_price'] = df_day['close'].mean()

# 一連の処理
def analyzer(ticker, df_result, index):
  json_day, json_minute = get_price_data_from_yahoo(ticker)
  if json_day is None or json_minute is None:
    return

  df_day, df_minute = convert_to_dataframe(json_day, json_minute)
  if df_day is None or df_minute is None:
    return

  analyze_direction(df_day, df_minute, index, df_result)

In [ ]:
%%monitor_cell 'Analyzer'

import codecs
import time
from IPython.display import display

#df_result = pd.read_csv('./tickers.csv')
# UTF-8のエラー対応のため上のコードの代わりに次のコードを実行し、銘柄コード一覧を取得
with codecs.open('./tickers.csv', 'r', 'UTF-8', 'ignore') as file:
  df_result = pd.read_table(file, delimiter=',')

# 東証前提のため銘柄コードの末尾に.Tを付与
df_result['ticker'] = list(map(str, df_result['ticker']))
df_result['ticker'] = df_result['ticker'] + '.T'

start = time.time()

for row in df_result.itertuples():
  analyzer(row.ticker, df_result, row[0])
  time.sleep(INTERVAL_SECOND)
  if row[0] % 100 == 99:
  # if row[0] % 10 == 3: # テスト用コード
    process_time = time.time() - start
    print(f'{str(row[0]+1)}社目のデータ取得後の結果（これまで{str(round(process_time/60, 1))}分経過）')
    display(df_result.sort_values('match_rate', ascending=False))

df_result = df_result.dropna(how='any')
if 'match_count' in df_result:
  df_result['match_count'] = list(map(int, df_result['match_count']))
if 'whole_count' in df_result:
  df_result['whole_count'] = list(map(int, df_result['whole_count']))

df_result.to_csv('./result.csv')

100社目のデータ取得後の結果（これまで8.9分経過）


,ticker,name,match_rate,match_count,whole_count,average_volume,average_price
22,1343.T,ＮＥＸＴ ＦＵＮＤＳ 東証ＲＥＩＴ指数連動型上場投信,0.733333,44.0,60.0,4.040960e+05,2119.800000
66,1417.T,ミライト・ホールディングス,0.716667,43.0,60.0,2.722600e+05,1779.483333
3,1308.T,上場インデックスファンドＴＯＰＩＸ,0.683333,41.0,60.0,3.693967e+05,2001.083333
2,1306.T,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,0.683333,41.0,60.0,2.540981e+06,2022.600000
29,1356.T,ＴＯＰＩＸベア２倍上場投信,0.666667,40.0,60.0,4.101577e+05,1201.583333
...,...,...,...,...,...,...,...
4091,9993.T,ヤマザワ,NaN,NaN,NaN,NaN,NaN
4092,9994.T,やまや,NaN,NaN,NaN,NaN,NaN
4093,9995.T,グローセル,NaN,NaN,NaN,NaN,NaN
4094,9996.T,サトー商会,NaN,NaN,NaN,NaN,NaN


200社目のデータ取得後の結果（これまで17.5分経過）


,ticker,name,match_rate,match_count,whole_count,average_volume,average_price
120,1491.T,中外鉱業,0.800000,48.0,60.0,2.115723e+06,33.483333
129,1514.T,住石ホールディングス,0.750000,45.0,60.0,2.925042e+06,120.750000
22,1343.T,ＮＥＸＴ ＦＵＮＤＳ 東証ＲＥＩＴ指数連動型上場投信,0.733333,44.0,60.0,4.040960e+05,2119.800000
66,1417.T,ミライト・ホールディングス,0.716667,43.0,60.0,2.722600e+05,1779.483333
3,1308.T,上場インデックスファンドＴＯＰＩＸ,0.683333,41.0,60.0,3.693967e+05,2001.083333
...,...,...,...,...,...,...,...
4091,9993.T,ヤマザワ,NaN,NaN,NaN,NaN,NaN
4092,9994.T,やまや,NaN,NaN,NaN,NaN,NaN
4093,9995.T,グローセル,NaN,NaN,NaN,NaN,NaN
4094,9996.T,サトー商会,NaN,NaN,NaN,NaN,NaN


300社目のデータ取得後の結果（これまで26.5分経過）


...

3900社目のデータ取得後の結果（これまで347.9分経過）


,ticker,name,match_rate,match_count,whole_count,average_volume,average_price
3743,9101.T,日本郵船,1.000000,1.0,1.0,5.146500e+06,4290.000000
223,1689.T,ＷｉｓｄｏｍＴｒｅｅ 天然ガス上場投資信託,0.883333,53.0,60.0,7.243331e+06,1.637931
3567,8628.T,松井証券,0.850000,51.0,60.0,7.736033e+05,915.900000
3607,8836.T,ＲＩＳＥ,0.816667,49.0,60.0,6.500667e+05,28.483333
2076,5809.T,タツタ電線,0.816667,49.0,60.0,2.595450e+05,640.350000
...,...,...,...,...,...,...,...
4091,9993.T,ヤマザワ,NaN,NaN,NaN,NaN,NaN
4092,9994.T,やまや,NaN,NaN,NaN,NaN,NaN
4093,9995.T,グローセル,NaN,NaN,NaN,NaN,NaN
4094,9996.T,サトー商会,NaN,NaN,NaN,NaN,NaN


4000社目のデータ取得後の結果（これまで356.7分経過）


,ticker,name,match_rate,match_count,whole_count,average_volume,average_price
3743,9101.T,日本郵船,1.000000,1.0,1.0,5.146500e+06,4290.000000
223,1689.T,ＷｉｓｄｏｍＴｒｅｅ 天然ガス上場投資信託,0.883333,53.0,60.0,7.243331e+06,1.637931
3567,8628.T,松井証券,0.850000,51.0,60.0,7.736033e+05,915.900000
3579,8714.T,池田泉州ホールディングス,0.816667,49.0,60.0,2.185317e+06,171.983333
3378,8107.T,キムラタン,0.816667,49.0,60.0,6.204183e+05,25.350000
...,...,...,...,...,...,...,...
4091,9993.T,ヤマザワ,NaN,NaN,NaN,NaN,NaN
4092,9994.T,やまや,NaN,NaN,NaN,NaN,NaN
4093,9995.T,グローセル,NaN,NaN,NaN,NaN,NaN
4094,9996.T,サトー商会,NaN,NaN,NaN,NaN,NaN


This run of 'Analyzer' ran for 6:05:05 and logs are available locally at: /root/.hyperdash/logs/analyzer/analyzer_2021-04-30t23-10-53-993684.log


# 3. Code Snippet

In [ ]:
# デバッガー
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [ ]:
# デバッグするときは次のコードをブレイクポイントに設定
from IPython.core.debugger import Pdb; Pdb().set_trace()

# ノートブックの稼働時間を調べたいときは次のコードを別ノートブックで実行
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'